In [ ]:
#Data Scrapping

In [ ]:

import time
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException

# Initialize the Chrome WebDriver
driver = webdriver.Chrome()

# Open the website
driver.get("")  #provide the link that you want to scrape
driver.maximize_window()            #maximize the window button
time.sleep(5)  # Wait for 5min the page to load

#some website ask the signup 
# Function to close the signup popup if it appears
def close_signup_popup():
    try:
        close_button = driver.find_element(By.CLASS_NAME, "icon-cross")
        driver.execute_script("arguments[0].click();", close_button)
        time.sleep(2)  # Wait for the popup to close
        print("Signup popup closed.")  #ensure it has closed
    except NoSuchElementException:
        # If the popup is not found, we just continue
        pass

# Locate the route element by its class name 'route'
def get_links_and_text():
    roots = driver.find_elements(By.CLASS_NAME, 'route')
    root_names = [i.text for i in roots]   #get the all route name through the loop
    links = [element.get_attribute('href') for element in roots]   #get the links based on the route name
    return root_names, links

#many website have more than one page 
def go_to_next_page():
    wait = WebDriverWait(driver, 10)
    try:
        #select the xpath in active page
        active_page_element = driver.find_element(By.XPATH, "//div[@class='DC_117_pageTabs DC_117_pageActive']")  
        active_page_number = active_page_element.text
        next_page_number = str(int(active_page_number) + 1)  #move to the next page by add 1 with current page value 
        
        next_page_button = wait.until(EC.element_to_be_clickable((By.XPATH, f"//div[@class='DC_117_paginationTable']//div[text()='{next_page_number}']")))
        next_page_button.click()  #you can move to the next page by clcicking specificic element
        time.sleep(5)  # Wait for the next page to load
    except Exception as e:
        print(f"Error while clicking next page: {e}")

# Retrieve data from all pages
all_root_names = []   #create the empty list 
all_links = []

for i in range():  # Assuming there are multiple pages
    root_names, links = get_links_and_text()
    all_root_names.extend(root_names)  #inser the all data my extend method instead of append
    all_links.extend(links)
    if i < :  # Adjust based on the number of pages.because ,the loop starts from 0,1,2...
        go_to_next_page()

# Navigate to each route and extract bus data
bus_data = []   #create a mpty list to store the data

for route_name, link in zip(all_root_names, all_links):  #the loop iterate the link based on route name, link
    driver.get(link)
    time.sleep(5)  # Wait for the page to load

    # Find all "View Buses" buttons
    #the view bus option contain some goverment bus details
    view_buses_buttons = driver.find_elements(By.CLASS_NAME, "button")

    for button in view_buses_buttons:
        try:
            #we can acess those data by clicking the "view bus" option
            driver.execute_script("arguments[0].click();", button)
            time.sleep(5)  # Wait for buses to load
            
            # Close the signup popup if it appears
            close_signup_popup()
            
            #some websites have scroll bar , to extend the page range for display more data
            #we can acess those data by scrolling the page
            # Scroll down to load more buses
            last_height = driver.execute_script("return document.body.scrollHeight") #javascript commends  1st time
            while True:
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(5)
                new_height = driver.execute_script("return document.body.scrollHeight")  #javascript commends  2st time
                if new_height == last_height:  #check the conditions are same 
                    break    #if it is true break the loop
                last_height = new_height #assign the values to each other

            # Extract bus details
            bus_elements = driver.find_elements(By.CLASS_NAME, 'row-sec')   #assign that division element

            for element in bus_elements:
                try:
                    bus_name = element.find_element(By.CLASS_NAME, 'travels').text.strip()  #busname element
                    bus_type = element.find_element(By.CLASS_NAME, 'bus-type').text.strip()  #bustype element
                    departing_time = element.find_element(By.CLASS_NAME, 'dp-time').text.strip()  #starting time element
                    duration = element.find_element(By.CLASS_NAME, 'dur').text.strip()         #duration elelemnt 
                    reaching_time = element.find_element(By.CLASS_NAME, 'bp-time').text.strip()  #reaching time element
                    star_rating = element.find_element(By.XPATH, './/div[@class="rating-sec lh-24"]//span').text.strip()  #star rating element by xpath
                    try:
                        price_element = element.find_element(By.CSS_SELECTOR, 'span.f-19.f-bold')  #price element by css selector
                        price = price_element.text.strip()
                    except NoSuchElementException:
                        price = "Price not available"

                    seats_available = element.find_element(By.CLASS_NAME, 'seat-left').text.strip().split()[0]  #seats element by selecting first index value

                    bus_data.append({    #assign the all data in dictionary structure by append method 
                        "Route Name": route_name,
                        "Route Link": link,
                        "Bus Name": bus_name,
                        "Bus Type": bus_type,
                        "Departing Time": departing_time,
                        "Duration": duration,
                        "Reaching Time": reaching_time,
                        "Star Rating": star_rating,
                        "Price": price,
                        "Seats Available": int(seats_available)
                    })
                except Exception as e:
                    print(f"Error extracting bus data: {e}")  # Print the error message to help with debugging and continue execution

        except Exception as e:
            print(f"Error clicking 'View Buses' button: {e}")  

# Save the extracted bus data to a CSV file
csv_file_path = "punjab.csv"  #set file name and extension
# Open the CSV file in write mode ('w') with specified newline and encoding settings
with open(csv_file_path, mode='w', newline='', encoding='utf-8') as file:
    # Create a CSV DictWriter object, specifying the fieldnames based on the keys of the first dictionary in bus_data
    writer = csv.DictWriter(file, fieldnames=bus_data[0].keys())
    writer.writeheader()   #take the first row as headr
    # Write all the rows of bus data to the CSV file
    writer.writerows(bus_data)

#ensure the file has been saved
print(f"Data has been saved to {csv_file_path}")

# Close the browser
time.sleep(5)
driver.quit()


In [ ]:

import pandas as pd

# Read the file with specified encoding
pj = pd.read_csv(r'D:\0.Projects -Guvi\punjab.csv', encoding='utf-8')

# Remove duplicate rows
pj_cleaned = pj.drop_duplicates()

# Save the cleaned data back to the same file, overwriting it
pj_cleaned.to_csv(r'D:\0.Projects -Guvi\punjab.csv', index=False, encoding='utf-8')

#ensure the duplicates are removed and the file has been updated
print("Duplicates removed and data saved.")
